# Luồng 0.1: Xây dựng Cơ sở dữ liệu Vector từ Schema

**Mục tiêu:** Script này sẽ thực hiện các công việc sau:
1.  Đọc và phân tích tệp `m-schema_final.txt` để trích xuất các mô tả (description) của bảng và cột.
2.  Sử dụng mô hình ngôn ngữ từ `sentence-transformers` để chuyển đổi các mô tả này thành vector embeddings.
3.  Lưu trữ các vector và metadata tương ứng vào một cơ sở dữ liệu vector (ChromaDB) trên đĩa.

**Thư viện cần thiết:** `chromadb`, `sentence-transformers`, `regex`.

In [1]:
import chromadb
from sentence_transformers import SentenceTransformer
import re
import os

# --- CONFIGURATION ---
SCHEMA_FILE_PATH = 'm-schema_final.txt'
VECTOR_DB_PATH = 'vector_database' # Thư mục để lưu trữ DB
COLLECTION_NAME = 'schema_descriptions'
MODEL_NAME = 'all-MiniLM-L6-v2' # Một mô hình phổ biến, nhẹ và hiệu quả

c:\Users\Lenovo\anaconda3\envs\t2sql\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Hàm Phân tích Schema

Hàm này chịu trách nhiệm đọc tệp schema và trích xuất thông tin cần thiết (mô tả bảng, mô tả cột).

In [2]:
def parse_schema_for_descriptions(file_path):
    """
    Parses the schema file to extract table and column descriptions.
    """
    print(f"Đang phân tích tệp schema: {file_path}")
    if not os.path.exists(file_path):
        print(f"LỖI: Không tìm thấy tệp {file_path}")
        return {}

    schema_info = {}
    current_table = None
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith('# Table:'):
                current_table = line.replace('# Table:', '').strip()
                schema_info[current_table] = {'description': '', 'columns': []}
            elif line.startswith('【Table Description】'):
                if current_table:
                    desc_text = line.replace('【Table Description】', '').strip().strip('"')
                    schema_info[current_table]['description'] = desc_text
            elif current_table and line.startswith('('):
                # Regex để trích xuất tên cột và mô tả từ định dạng: (column_name:TYPE, "Description.", ...)
                match = re.search(r'\((?P<name>\w+):.*?, "(?P<desc>.*?)"', line)
                if match:
                    col_name = match.group('name')
                    col_desc = match.group('desc')
                    schema_info[current_table]['columns'].append({
                        'name': col_name,
                        'description': col_desc
                    })
    print("Phân tích schema hoàn tất.")
    return schema_info

## 3. Logic Chính: Xây dựng và Lưu trữ Database

Cell này chứa logic chính để thực hiện quá trình xây dựng DB vector. Nó sẽ:
1.  Gọi hàm `parse_schema`.
2.  Khởi tạo ChromaDB client.
3.  Nạp mô hình `sentence-transformer`.
4.  Chuẩn bị dữ liệu (documents, metadatas, ids).
5.  Tạo embeddings và thêm vào collection của ChromaDB.

In [3]:
# 1. Parse schema file
schema_data = parse_schema_for_descriptions(SCHEMA_FILE_PATH)

if schema_data:
    # 2. Initialize ChromaDB client and collection
    client = chromadb.PersistentClient(path=VECTOR_DB_PATH)
    collection = client.get_or_create_collection(name=COLLECTION_NAME)
    print(f"ChromaDB client đã khởi tạo. Collection '{COLLECTION_NAME}' đã sẵn sàng.")

    # 3. Load the embedding model
    print(f"Đang tải mô hình sentence-transformer: '{MODEL_NAME}'...")
    model = SentenceTransformer(MODEL_NAME)
    print("Tải mô hình thành công.")

    # 4. Prepare data for embedding
    documents = []
    metadatas = []
    ids = []
    id_counter = 1

    for table_name, info in schema_data.items():
        if info['description']:
            documents.append(info['description'])
            metadatas.append({'source': 'table_description', 'table': table_name})
            ids.append(f"doc_{id_counter}")
            id_counter += 1
            
        for col in info['columns']:
            if col['description']:
                doc_text = f"Column '{col['name']}' in table '{table_name}': {col['description']}"
                documents.append(doc_text)
                metadatas.append({'source': 'column_description', 'table': table_name, 'column': col['name']})
                ids.append(f"doc_{id_counter}")
                id_counter += 1

    print(f"Đã chuẩn bị {len(documents)} văn bản để tạo embedding.")

    # 5. Generate embeddings and add to collection
    if documents:
        print("Đang tạo embeddings cho tất cả văn bản...")
        embeddings = model.encode(documents).tolist()
        print("Tạo embeddings hoàn tất.")
        
        # Xóa collection cũ để đảm bảo dữ liệu mới nhất
        client.delete_collection(name=COLLECTION_NAME)
        collection = client.create_collection(name=COLLECTION_NAME)

        # Thêm dữ liệu vào collection
        collection.add(
            ids=ids,
            embeddings=embeddings,
            documents=documents,
            metadatas=metadatas
        )
        print("\n--- Xây dựng Cơ sở dữ liệu Vector HOÀN TẤT! ---")
        print(f"Tổng số documents đã được lập chỉ mục: {collection.count()}")
        print(f"Cơ sở dữ liệu đã được lưu tại: {os.path.abspath(VECTOR_DB_PATH)}")
    else:
        print("Không có văn bản mô tả nào để lập chỉ mục.")

Đang phân tích tệp schema: m-schema_final.txt
Phân tích schema hoàn tất.
ChromaDB client đã khởi tạo. Collection 'schema_descriptions' đã sẵn sàng.
Đang tải mô hình sentence-transformer: 'all-MiniLM-L6-v2'...
Tải mô hình thành công.
Đã chuẩn bị 78 văn bản để tạo embedding.
Đang tạo embeddings cho tất cả văn bản...
Tạo embeddings hoàn tất.

--- Xây dựng Cơ sở dữ liệu Vector HOÀN TẤT! ---
Tổng số documents đã được lập chỉ mục: 78
Cơ sở dữ liệu đã được lưu tại: c:\Users\Lenovo\OneDrive\Máy tính\simple-chicken\PythonTextToSql\Method\Offline\build_vector_db\vector_database
